Klasyfikacja uporządkowanych danych za pomocą warstw przetwarzania wstępnego

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [2]:
#ładowanie zbioru danych petfinder
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip',dataset_url,extract=True,cache_dir='.')
dataframe = pd.read_csv(csv_file)

1668792/1668792 [==============================] - 1s 0us/step


In [3]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [4]:
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4,0,1)
dataframe = dataframe.drop(columns=['AdoptionSpeed','Description'])

In [5]:
train,val,test = np.split(dataframe.sample(frac=1),[int(0.8*len(dataframe)),int(0.9*len(dataframe))])

In [6]:
print(len(train),'training examples')
print(len(val),'validation examples')
print(len(test),'test examples')

9229 training examples
1154 validation examples
1154 test examples


In [7]:
#utworzenie potoku wejściowego
def df_to_dataset(dataframe,shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key:value[:,tf.newaxis] for key,value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df),labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [8]:
batch_size = 5
train_ds = df_to_dataset(train,batch_size = batch_size)

<ipython-input-7-005602ec104a>:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key:value[:,tf.newaxis] for key,value in dataframe.items()}


In [9]:
[(train_features,label_batch)] = train_ds.take(1)
print('każdy przypadek:',list(train_features.keys()))
print('Batch -> ages:',train_features['Age'])
print('Batch dla celów:',label_batch)

każdy przypadek: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
Batch -> ages: tf.Tensor(
[[ 2]
 [ 6]
 [ 6]
 [ 3]
 [48]], shape=(5, 1), dtype=int64)
Batch dla celów: tf.Tensor([1 1 1 1 1], shape=(5,), dtype=int64)
